YouTube Link: https://www.youtube.com/watch?v=QcKHfMgcnbw

In [8]:
# permutate
P10 = [
    (1, 3),
    (2, 5),
    (3, 2),
    (4, 7),
    (5, 4),
    (6, 10),
    (7, 1),
    (8, 9),
    (9, 8),
    (10, 6)
]

In [9]:
# select and permutate
P8 = [
    (1, 6),
    (2, 3),
    (3, 7),
    (4, 4),
    (5, 8),
    (6, 5),
    (7, 10),
    (8, 9)
]

In [10]:
# permutate
P4 = [
    (1, 2),
    (2, 4),
    (3, 3),
    (4, 1)
]

In [32]:
IP = [
    (1, 2),
    (2, 6),
    (3, 3),
    (4, 1),
    (5, 4),
    (6, 8),
    (7, 5),
    (8, 7)
]

In [44]:
EP = [
    (1, 4),
    (2, 1),
    (3, 2),
    (4, 3),
    (5, 2),
    (6, 3),
    (7, 4),
    (8, 1)
]

##### Get Key and Plain Text

In [37]:
# get key
key = '1010000010'
plain_text = '01110010'

##### Generate Subkeys

###### 1. Apply P10

In [38]:
def apply_p10(key: str) -> str:
    nkey = [0] * 10
    for f, s in P10:
        nkey[f-1] = key[s-1]  # zero based indexing
    nkey = "".join(nkey)
    return nkey

nkey = apply_p10(key)
print(nkey)

1000001100


###### 2. Split into two halfs and apply one-bit left shift to each

In [39]:
def split_half(key: str) -> tuple:
    n = len(key)
    mid = n//2
    return key[:mid], key[mid:]

def left_shift(key: str, shift=1) -> str:
    n = len(key)
    shift = shift%n
    key = key[shift:] + key[:shift]  # left side moves to the end
    return key

first, second = split_half(nkey)
ls1 = left_shift(first) + left_shift(second)
print(ls1)

0000111000


###### 3. Apply P8 to get K1

In [40]:
def apply_p8(key: str) -> str:
    nkey = [0] * 8
    for f, s in P8:
        nkey[f-1] = key[s-1]  # zero based indexing
    nkey = "".join(nkey)
    return nkey

k1 = apply_p8(ls1)
print(k1)

10100100


###### 4. Split half (ls1) and perform two-bit left shift to each 

In [41]:
first, second = split_half(ls1)
ls2 = left_shift(first, shift=2) + left_shift(second, shift=2)
print(ls2)

0010000011


###### 5. Apply P8 to get K2

In [42]:
k2 = apply_p8(ls2)
print(k2)

01000011


##### Encryption

###### 1. Apply IP

In [43]:
def apply_ip(pt: str) -> str:
    npt = [0] * 8
    for f, s in IP:
        npt[f-1] = pt[s-1]  # zero based indexing
    npt = "".join(npt)
    return npt

ip = apply_ip(plain_text)
print(ip)

10101001


###### 2. Split into two halfs and use second half for further

In [78]:
first_ip, second_ip = split_half(ip)
print(first_ip, second_ip)

1010 1001


In [79]:
def apply_ep(pt: str) -> str:
    npt = [0] * len(EP)
    for f, s in EP:
        npt[f-1] = pt[s-1]
    npt = "".join(npt)
    return npt

ep = apply_ep(second_ip)
print(ep)

11000011


In [80]:
def xor(first, second):
    result = []
    for f, s in zip(first, second):
        result.append(int(f) ^ int(s))
    result = [str(x) for x in result]
    result = "".join(result)
    return result

xor_with_k1 = xor(ep, k1)
print(xor_with_k1)

01100111


In [81]:
s0 = [
    ['01', '00', '11', '10'],
    ['11', '10', '01', '00'],
    ['00', '10', '01', '11'],
    ['11', '01', '11', '10']
]

s1 = [
    ['00', '01', '10', '11'],
    ['10', '00', '01', '11'],
    ['11', '00', '01', '00'],
    ['10', '01', '00', '11']
]

In [82]:
# row is (b1, b4) and col is (b2, b3)
def feed_to_s0(pt):
    row = int(pt[0]+pt[3], base=2)
    col = int(pt[1]+pt[2], base=2)
    
    return s0[row][col]

def feed_to_s1(pt):
    row = int(pt[0]+pt[3], base=2)
    col = int(pt[1]+pt[2], base=2)
    
    return s1[row][col]  

In [83]:
# split and feed xor_with_k1 to s0 and s1
first, second = split_half(xor_with_k1)
sub_result = feed_to_s0(first) + feed_to_s1(second)
print(sub_result)

1011


In [84]:
# apply p4 and then xor with left half of ip
def apply_p4(pt: str) -> str:
    npt = [0] * len(P4)
    for f, s in P4:
        npt[f-1] = pt[s-1]
    npt = "".join(npt)
    return npt

In [85]:
result = apply_p4(sub_result)
print(result)

0111


In [86]:
xor_with_left_half = xor(result, first_ip)
print(xor_with_left_half)

1101


In [87]:
# combine this with right half
first_round_result = xor_with_left_half + second_ip
print(first_round_result)

11011001


In [88]:
# now swap the halfs and repeat
def swap_half(pt: str) -> str:
    first, second = split_half(pt)
    return second + first

In [96]:
swap_result = swap_half(first_round_result)
first_ip, second_ip = split_half(swap_result)
# use second ip further
ep = apply_ep(second_ip)
xor_with_k2 = xor(ep, k2)
first, second = split_half(xor_with_k2)
sub_result = feed_to_s0(first) + feed_to_s1(second)
result = apply_p4(sub_result)
xor_with_left_half = xor(result, first_ip)
second_round_result = xor_with_left_half + second_ip
print(second_round_result)

11101101


In [99]:
def apply_inverse_ip(pt: str) -> str:
    npt = [0] * len(IP)
    for f, s in IP:
        npt[s-1] = pt[f-1]  # inverse operation
    npt = "".join(npt)
    return npt

In [100]:
cipher_text = apply_inverse_ip(second_round_result)
print(cipher_text)

01110111


<img src="https://www.codeproject.com/KB/security/SDES/SDES_fig_C1.png" height=400 width=400>

In [108]:
def generate_subkeys(key: str) -> tuple:
    key = apply_p10(key)
    first, second = split_half(key)
    ls1 = left_shift(first, shift=1) + left_shift(second, shift=1)
    k1 = apply_p8(ls1)
    first, second = split_half(ls1)
    ls2 = left_shift(first, shift=2) + left_shift(second, shift=2)
    k2 = apply_p8(ls2)
    
    return k1, k2

In [109]:
def fK(ip: str, key: str) -> str:
    first_ip, second_ip = split_half(ip)
    
    # use second ip further
    ep = apply_ep(second_ip)
    xor_with_key = xor(ep, key)
    first, second = split_half(xor_with_key)
    sub_result = feed_to_s0(first) + feed_to_s1(second)
    result = apply_p4(sub_result)
    xor_with_left_half = xor(result, first_ip)
    result = xor_with_left_half + second_ip

    return result

In [110]:
def encryption(plain_text: str, key: str) -> str:
    k1, k2 = generate_subkeys(key)
    
    # round 1
    ip = apply_ip(plain_text)
    fk1 = fK(ip, k1)
    sw = swap_half(fk1)
    fk2 = fK(sw, k2)
    cipher_text = apply_inverse_ip(fk2)
    
    return cipher_text

In [111]:
key = '1010000010'
plain_text = '01110010'

In [115]:
cipher_text = encryption(plain_text, key)
print(cipher_text)

01110111


In [116]:
def decryption(cipher_text: str, key: str) -> str:
    k1, k2 = generate_subkeys(key)
    ip = apply_ip(cipher_text)
    fk2 = fK(ip, k2)
    sw = swap_half(fk2)
    fk1 = fK(sw, k1)
    plain_text = apply_inverse_ip(fk1)
    
    return plain_text

In [117]:
decrypted_text = decryption(cipher_text, key)
print(decrypted_text)

01110010


In [122]:
assert decrypted_text == plain_text